# Agents Semantic Kernel en Python

Dans ce notebook, nous allons :
1. Installer / Importer **semantic-kernel** en Python
2. Illustrer la création d'un agent simple (single agent) et son invocation
3. Illustrer un **group chat** (AgentGroupChat) avec sélection/termination
4. Conclusion

In [1]:
# ============================
# Bloc 1 : Installation semantic-kernel et imports
# ============================

# À n’exécuter qu'une fois
%pip install semantic-kernel --quiet
import asyncio
import logging

print("semantic-kernel installé.")


Note: you may need to restart the kernel to use updated packages.
semantic-kernel installé.


## Bloc 2 : Simple Agent (Parrot)

Nous créons un agent tout simple, qui répète le message de l’utilisateur sur le ton d’un pirate.

In [2]:
import logging
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory

AGENT_NAME = "Parrot"
AGENT_INSTRUCTIONS = "You are a helpful parrot that repeats the user message in a pirate voice, then ends with 'Arrr!'"

# Création du Kernel
kernel = Kernel()
# On suppose que vous avez défini ou récupéré des clés d'API :
# kernel.add_service(OpenAIChatCompletion(...)) ou AzureChatCompletion(...)
kernel.add_service(OpenAIChatCompletion(service_id="agent"))

agent = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS
)
user_inputs = [
    "Fortune favors the bold.",
    "I came, I saw, I conquered.",
    "Practice makes perfect.",
]

async def simple_agent_demo():
    chat_history = ChatHistory()
    # On ajoute les instructions de l'agent en tant que 'developer' ou 'system'
    chat_history.add_developer_message(AGENT_INSTRUCTIONS)

    for user_input in user_inputs:
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")
        async for content in agent.invoke(chat_history):
            chat_history.add_message(content)
            print(f"# Agent - {content.name or '*'}: '{content.content}'")

await simple_agent_demo()


# User: 'Fortune favors the bold.'
# Agent - Parrot: 'Fortune favors the bold, arrr!'
# User: 'I came, I saw, I conquered.'
# Agent - Parrot: 'I came, I saw, I conquered, arrr!'
# User: 'Practice makes perfect.'
# Agent - Parrot: 'Practice makes perfect, arrr!'


## Bloc 3 : Agent simple avec Plugins

Exemple de plugin `MenuPlugin`, et agent unique qui répond en utilisant ces fonctions.


In [3]:
import asyncio
from typing import TYPE_CHECKING, Annotated
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments, kernel_function
from semantic_kernel.contents import ChatHistory

class MenuPlugin:
    """Plugin pour gérer un menu"""
    @kernel_function(description="Liste les specials")
    def get_specials(self) -> Annotated[str, "Describes specials"]:
        return "Special Soup: Clam Chowder\nSpecial Salad: Cobb Salad\nSpecial Drink: Chai Tea"
    @kernel_function(description="Donne le prix d'un item")
    def get_item_price(self, menu_item: Annotated[str, "nom de l'item"]) -> str:
        return "$9.99"
# Créer kernel
kernel2 = Kernel()
# Ajout du plugin
kernel2.add_plugin(MenuPlugin(), plugin_name="menu")
# Ajout du service
kernel2.add_service(OpenAIChatCompletion(service_id="agent2"))
# On configure l'auto function-calling
settings2 = kernel2.get_prompt_execution_settings_from_service_id(service_id="agent2")
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
settings2.function_choice_behavior = FunctionChoiceBehavior.Auto()

AGENT2_NAME = "Host"
AGENT2_INSTRUCTIONS = "Answer questions about the menu."
agent2 = ChatCompletionAgent(
    service_id="agent2",
    kernel=kernel2,
    name=AGENT2_NAME,
    instructions=AGENT2_INSTRUCTIONS,
    arguments=KernelArguments(settings=settings2),
)
async def plugin_agent_demo():
    chat_history = ChatHistory()
    user_msgs = [
        "Hello",
        "What is the special soup?",
        "What does it cost?",
        "Thanks",
    ]
    for user_input in user_msgs:
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")
        agent_name = None
        async for content in agent2.invoke_stream(chat_history):
            if not agent_name:
                agent_name = content.name or AGENT2_NAME
                print(f"# {agent_name}: '", end="")
            if content.content.strip():
                print(content.content, end="", flush=True)
        print("'")

await plugin_agent_demo()


# User: 'Hello'
# Host: 'Hello! How can I assist you today?'
# User: 'What is the special soup?'
# Host: 'The special soup is Clam Chowder. Would you like to know more about it?'
# User: 'What does it cost?'
# Host: 'The Clam Chowder costs $9.99. Would you like to order it or need help with anything else?'
# User: 'Thanks'
# Host: 'You're welcome! If you have any more questions or need assistance, feel free to ask. Have a great day!'


## Bloc 4 : Group Chat

Exemple d'un chat groupé : un agent CopyWriter, un agent ArtDirector, etc. On utilise la `AgentGroupChat`.


In [4]:
import asyncio
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel import Kernel

class ApprovalTerminationStrategy(TerminationStrategy):
    async def should_agent_terminate(self, agent, history):
        return "approved" in history[-1].content.lower()

# On crée un kernel par agent, ou le même kernel + service differencié.
def create_kernel_for(name):
    k = Kernel()
    # on admet qu'on a paramétré un service openAI.
    k.add_service(OpenAIChatCompletion(service_id=name))
    return k

REVIEWER_NAME = "ArtDirector"
REVIEWER_INSTRUCTIONS = "You are an art director. If the copy is good, say 'Approved'. Otherwise, propose improvements."
reviewer_agent = ChatCompletionAgent(
    service_id=REVIEWER_NAME,
    kernel=create_kernel_for(REVIEWER_NAME),
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
)
COPYWRITER_NAME = "CopyWriter"
COPYWRITER_INSTRUCTIONS = "You are a copywriter. Provide short but strong marketing copy."
writer_agent = ChatCompletionAgent(
    service_id=COPYWRITER_NAME,
    kernel=create_kernel_for(COPYWRITER_NAME),
    name=COPYWRITER_NAME,
    instructions=COPYWRITER_INSTRUCTIONS,
)
group_chat = AgentGroupChat(
    agents=[reviewer_agent, writer_agent],
    termination_strategy=ApprovalTerminationStrategy(agents=[reviewer_agent], maximum_iterations=6)
)

async def group_chat_demo():
    user_msg = "I need a slogan for a new line of electric bikes"
    await group_chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_msg))
    print(f"# User: '{user_msg}'")
    async for content in group_chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")

    print(f"# IS COMPLETE: {group_chat.is_complete}")

await group_chat_demo()


# User: 'I need a slogan for a new line of electric bikes'
# Agent - ArtDirector: 'Sure! Here are a few options for your electric bike line:

1. "Ride the Future."
2. "Power Your Ride."
3. "Unleash the Electric!"
4. "Revolutionize Your Commute."
5. "Energize Your Journey."

Please let me know if you would like to explore more options or need any adjustments!'
# Agent - CopyWriter: 'Here are some catchy slogans for your electric bike line:

1. "Charge Ahead."
2. "Eco-Freedom on Two Wheels."
3. "Electrify Your Ride."
4. "Pedal Less, Explore More."
5. "Go Green, Ride Clean."

Feel free to ask for more suggestions or variations!'
# Agent - ArtDirector: 'Approved.'
# IS COMPLETE: True


## Conclusion

Nous avons illustré **plusieurs scénarios** d’agents en Python avec Semantic Kernel :
- Un agent **unique** type “parrot” qui répète le user input.
- Un agent **unique** + plugins (function-calling auto-invoqué).
- Un **group chat** d’agents (ex: CopyWriter, ArtDirector), orchestré via `AgentGroupChat`.

Vous pouvez adapter ces exemples à vos propres clés API (OpenAI vs Azure), configurer l’autogestion des tools, ou enrichir avec vos plugins sémantiques.
Et voilà un “notebook 3” purement Python côté Semantic Kernel. 😉